In [1]:
import pandas as pd 
import numpy as np 
import os

# Data Organization

In [2]:
os.listdir('./../data/')

['H190923', 'H200210', 'H201120', 'HEK8', 'aggregated_data.csv']

In [3]:
os.listdir('./../data/H201120/')

['SKBR3 viability_72h treatment.xlsx',
 'clover_all_cell.csv',
 'mscarlet_all_cell.csv']

In [4]:
viab1 = pd.read_excel('./../data/H190923/H100923_K562_viability.xlsx', engine='openpyxl')
viab1.head()

,Un,Das_0.625,Das_1.25,Das_2.5,Das_5,Das_10,Das_100,Das_1000,10uM_Tramet,10uM_MK2206
0,15990000,6819000,2979000,815200,409700,421900,294000,738600,4788000,40070
1,15900000,6793000,2749000,704000,396700,385800,252500,702000,4825000,38480
2,16790000,6735000,2869000,725000,337100,363200,263600,730300,4521000,44160


In [5]:
#viab2 = pd.read_excel('./../data/H201120/SKBR3 viability_72h treatment.xlsx', engine='openpyxl')
#viab2.head(10)

In [6]:
#viab3 = pd.read_excel('./../data/H200210/Viability.xlsx', engine='openpyxl')
#viab3.head(10)

In [7]:
def load_pathway_data(clover_path, mscarlet_path, folder): 
    '''
    '''    
    # load clover data 
    clover = pd.read_csv(clover_path)
    if ('site' in clover.columns): clover = clover.drop(['site'], axis=1)    
    if ('start' in clover.columns): clover = clover.drop(['start'], axis=1)    
    clover = clover.set_index(['track_index', 'cell__treatment', 'median']).stack().reset_index()
    clover = clover.rename({'level_3':'time(h)', 0:'pathway_score'}, axis=1)
    clover = clover.assign(color='clover')
    clover = clover.assign(dataset=folder)
    
    # load mscarlet
    mscarlet = pd.read_csv(mscarlet_path)
    if ('site' in mscarlet.columns): mscarlet = mscarlet.drop(['site'], axis=1)
    if ('start' in mscarlet.columns): mscarlet = mscarlet.drop(['start'], axis=1)
    mscarlet = mscarlet.set_index(['track_index', 'cell__treatment', 'median']).stack().reset_index()
    mscarlet = mscarlet.rename({'level_3':'time(h)', 0:'pathway_score'}, axis=1)
    mscarlet = mscarlet.assign(color='mscarlet')
    mscarlet = mscarlet.assign(dataset=folder)
    
    # merge datasets 
    dat = pd.concat([clover, mscarlet], axis=0, ignore_index=True)

    return dat

In [8]:
data = []
for f in [x for x in os.listdir('./../data/') if x[-4:] != '.csv']: 
    data.append(load_pathway_data(f'./../data/{f}/clover_all_cell.csv', f'./../data/{f}/mscarlet_all_cell.csv', f))
    
data = pd.concat(data, axis=0, ignore_index=True)
data = data.reset_index(drop=True)
data.head()

,track_index,cell__treatment,median,time(h),pathway_score,color,dataset
0,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.0,0.699047,clover,H190923
1,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.2,0.860223,clover,H190923
2,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.5,0.833703,clover,H190923
3,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.8,0.876799,clover,H190923
4,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,1.0,0.744755,clover,H190923


In [9]:
def parse1(ct): 
    other, treatment = ct.split('__')
    line, path1, path2 = other.split('_')
    if treatment == 'untreated': 
        conc = None 
    else: 
        if ' ' in treatment: 
            conc, treatment = treatment.split(' ', 1)
        elif '_' in treatment: 
            conc, treatment = treatment.split('_', 1)
        else: 
            conc=None
    return line, treatment, conc, path1, path2
            
def parse2(ct): 
    _, other = ct.split('__')
    line, treatment = other.split('_', 1)
    
    if treatment == 'untreated': 
        conc = None 
    else: 
        if ' ' in treatment: 
            conc, treatment = treatment.split(' ', 1)
        elif '_' in treatment: 
            conc, treatment = treatment.split('_', 1)
        else: 
            conc=None
    return line, treatment, conc, None, None
    

temp = {'cell_line':[], 'pathway_1':[], 'pathway_2':[], 'inhibitor':[], 'conc':[]}
for ct in data.cell__treatment.values: 
    try: 
        if ct[4:6] == '__': 
            line, treatment, conc, path1, path2 = parse2(ct)
        else: 
            line, treatment, conc, path1, path2 = parse1(ct)

        temp['cell_line'].append(line)
        temp['pathway_1'].append(path1)
        temp['pathway_2'].append(path2)
        temp['inhibitor'].append(treatment)
        temp['conc'].append(conc)
    except: 
        print(ct)
        print(ct[4:6])
        raise


temp = pd.DataFrame(temp)
temp.head()

,cell_line,pathway_1,pathway_2,inhibitor,conc
0,k562,erk,akt,untreated,None
1,k562,erk,akt,untreated,None
2,k562,erk,akt,untreated,None
3,k562,erk,akt,untreated,None
4,k562,erk,akt,untreated,None


In [10]:
# remove any non-numeric values in `time(h)` ~ 'start' is in there
#data = data[~data['time(h)'].isin(['start', 'site'])]
#data = data.dropna()

data['time(h)'] = data['time(h)'].astype(float)
#data.head()

In [11]:
data = pd.concat([data, temp], axis=1)
data.head()

,track_index,cell__treatment,median,time(h),pathway_score,color,dataset,cell_line,pathway_1,pathway_2,inhibitor,conc
0,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.0,0.699047,clover,H190923,k562,erk,akt,untreated,None
1,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.2,0.860223,clover,H190923,k562,erk,akt,untreated,None
2,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.5,0.833703,clover,H190923,k562,erk,akt,untreated,None
3,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,0.8,0.876799,clover,H190923,k562,erk,akt,untreated,None
4,82_1000111247_k562_erk_akt__untreated,k562_erk_akt__untreated,0.890648,1.0,0.744755,clover,H190923,k562,erk,akt,untreated,None


In [20]:
data.to_csv('./../data/aggregated_data.csv', index=False)

# Exploratory Data Analysis 

In [13]:
data.describe()

,median,time(h),pathway_score
count,827509.000000,827509.000000,827509.000000
mean,0.417552,16.001022,0.423952
std,0.187658,14.970182,0.238399
min,0.021168,0.000000,-0.005041
25%,0.269632,5.500000,0.234176
50%,0.416500,11.200000,0.409203
75%,0.558281,20.200000,0.595467
max,0.948711,60.500000,1.008029


## double-check we got the time values right

In [14]:
data['time(h)'].unique()

array([ 0. ,  0.2,  0.5,  0.8,  1. ,  1.2,  1.5,  1.8,  2. ,  2.2,  2.5,
        2.8,  3. ,  3.2,  3.5,  3.8,  4. ,  4.2,  4.5,  4.8,  5. ,  5.2,
        5.5,  5.8,  6. ,  6.2,  6.5,  6.8,  7. ,  7.2,  7.5,  7.8,  8. ,
        8.2,  8.5,  8.8,  9. ,  9.2,  9.5,  9.8, 10. , 10.2, 10.5, 10.8,
       11. , 11.2, 11.5, 11.8, 12. , 12.2, 12.5, 12.8, 13. , 13.2, 13.5,
       13.8, 14. , 14.2, 14.5, 14.8, 15. , 15.2, 15.5, 15.8, 16. , 16.2,
       16.5, 16.8, 17. , 17.2, 17.5, 17.8, 18. , 18.2, 18.5, 18.8, 19. ,
       19.2, 19.5, 19.8, 20. , 20.2, 20.5, 20.8, 21. , 21.2, 21.5, 21.8,
       22. , 22.2, 22.5, 22.8, 23. , 23.2, 23.5, 23.8, 24. , 24.2, 24.5,
       24.8, 25. , 25.2, 25.5, 25.8, 26. , 26.2, 26.5, 26.8, 27. , 27.2,
       27.5, 27.8, 28. , 28.2, 28.5, 28.8, 29. , 29.2, 29.5, 29.8, 30. ,
       30.2, 30.5, 30.8, 31. , 31.2, 31.5, 31.8, 32. , 32.2, 32.5, 32.8,
       33. , 33.2, 33.5, 33.8, 34. , 34.2, 34.5, 34.8, 35. , 35.2, 35.5,
       35.8, 36. , 36.2, 36.5, 36.8, 37. , 37.2, 37

In [15]:
len(data['time(h)'].unique())

243

## How many treatment groups? 

Specifically, drugs + untreated 

In [16]:
data['inhibitor'].unique()

array(['untreated', 'dasatinib', 'trametinib', 'mk2206', 'bmn673',
       'etoposide', 'neratinib', 'lapatinib', 'ml_trastuzumab',
       'bmn673+trametinib'], dtype=object)

## How many concentrations for each drug? 

In [17]:
data.groupby('inhibitor')['conc'].unique()

inhibitor
bmn673                                                     [1um, None]
bmn673+trametinib                                               [None]
dasatinib            [0.625nm, 1.25nm, 2.5nm, 5nm, 10nm, 100nm, 100...
etoposide                                                  [1um, None]
lapatinib                                           [10nm, 100nm, 1um]
mk2206                                                          [10um]
ml_trastuzumab                                      [1ug, 10ug, 100ug]
neratinib                                           [1nm, 10nm, 100nm]
trametinib                                                [10um, None]
untreated                                                       [None]
Name: conc, dtype: object

## which pathways are measured? 

In [18]:
data.groupby(['pathway_1', 'pathway_2', 'inhibitor', 'cell_line'])['track_index'].count().reset_index().rename({'track_index':'count'}, axis=1)

,pathway_1,pathway_2,inhibitor,cell_line,count
0,cdk2,mdmx,bmn673,heya8,23664
1,cdk2,mdmx,bmn673+trametinib,heya8,26622
2,cdk2,mdmx,etoposide,heya8,10875
3,cdk2,mdmx,trametinib,heya8,31146
4,cdk2,mdmx,untreated,heya8,76734
5,erk,akt,dasatinib,k562,206416
6,erk,akt,lapatinib,skbr3,84564
7,erk,akt,mk2206,k562,15064
8,erk,akt,ml_trastuzumab,skbr3,92826
9,erk,akt,neratinib,skbr3,77760


## do all `cell__treatment`s have the same time points measured?

Looks like they do. 62 time points for each. 

In [19]:
__ = data.groupby(['cell__treatment', 'color'])['time(h)'].agg(['nunique', 'max', 'min'])#.reset_index().assign(ntime = lambda x: len(x['time(h)']))
#for i, row in __.iterrows():
#    print(row)
__

nunique   max  min
cell__treatment                    color                       
heya8_cdk2_mdmx__bmn673            clover         87  21.5  0.0
                                   mscarlet       87  21.5  0.0
heya8_cdk2_mdmx__bmn673+trametinib clover         87  21.5  0.0
                                   mscarlet       87  21.5  0.0
heya8_cdk2_mdmx__etoposide         clover         87  21.5  0.0
...                                              ...   ...  ...
skbr3_erk_akt__1ug_ml_trastuzumab  mscarlet      243  60.5  0.0
skbr3_erk_akt__1um_lapatinib       clover        243  60.5  0.0
                                   mscarlet      243  60.5  0.0
skbr3_erk_akt__untreated           clover        243  60.5  0.0
                                   mscarlet      243  60.5  0.0

[62 rows x 3 columns]

## For each track_index, there should be two colors (pathway measurements)

For some reason, there are some with only 1 pathway or the other... TODO

In [28]:
data.groupby('track_index')['color'].agg(['nunique', 'unique']).reset_index().groupby('nunique').count()

,track_index,unique
nunique,,
1,4707,4707
2,2231,2231
